In [1]:
!python -m venv myenv


In [3]:
!pip install -r requirements.txt

In [6]:
!aws s3 ls --profile ccprofile

2023-12-16 12:32:52 amplify-photoapp-staging-173239-deployment
2021-12-29 08:41:30 aws-codestar-us-east-1-052552561229
2021-12-29 08:42:08 aws-codestar-us-east-1-052552561229-test-pipe
2022-01-08 09:33:48 aws-sam-cli-managed-default-samclisourcebucket-18d75byvsmhla
2021-10-31 00:32:21 aws-sam-cli-managed-default-samclisourcebucket-cda3h9le376y
2024-10-13 21:50:48 datagardeners-privacy
2021-12-02 11:12:57 ideastorm-bot-python
2021-04-30 22:39:09 ideastorm-bucket
2024-10-14 00:02:49 privacy.datagardeners.ai
2024-05-01 12:21:56 sagemaker-studio-052552561229-xvzbr6r7pvo
2021-12-02 11:13:50 sagemaker-studio-07ujk3wwqh9l
2024-05-01 12:21:58 sagemaker-us-east-1-052552561229
2022-11-05 21:10:36 sam-app-dev-pipelineartifactsbucket-vzxn3ld8taft
2022-11-05 21:10:12 sam-app-dev-pipelineartifactsloggingbucket-1ky5tskm1ghdt


In [7]:
import duckdb
import pandas as pd

In [14]:
con = duckdb.connect('cc_index.duckdb')
con.install_extension("aws")
con.load_extension("aws")

con.install_extension("s3")
con.load_extension("s3")

con.install_extension("httpfs")
con.load_extension("httpfs")

In [15]:
CRAWL_SET = "CC-MAIN-2024-42"
CC_URL_S3_BASE="s3://"
CC_URL_HTTP_BASE="https://data.commoncrawl.org/"
CC_INDEX_URL = f"{CC_URL_HTTP_BASE}commoncrawl/cc-index/table/cc-main/warc/crawl={CRAWL_SET}/subset=warc/*.parquet"
CC_INDEX_S3_URL = f"{CC_URL_S3_BASE}commoncrawl/cc-index/table/cc-main/warc/crawl={CRAWL_SET}/subset=warc/*.parquet"
CC_INDEX_S3_BASE_URL = f"{CC_URL_S3_BASE}commoncrawl/cc-index/table/cc-main/warc/crawl={CRAWL_SET}/subset=warc/"
CC_INDEX_URL

'https://data.commoncrawl.org/commoncrawl/cc-index/table/cc-main/warc/crawl=CC-MAIN-2024-42/subset=warc/*.parquet'

In [16]:
CC_INDEX_S3_URL

's3://commoncrawl/cc-index/table/cc-main/warc/crawl=CC-MAIN-2024-42/subset=warc/*.parquet'

In [17]:
print(f"{CC_INDEX_S3_BASE_URL}")

s3://commoncrawl/cc-index/table/cc-main/warc/crawl=CC-MAIN-2024-42/subset=warc/


In [18]:
# Configure S3 settings for DuckDB
con.execute("""
    CALL load_aws_credentials('ccprofile');
""")

In [19]:
con.execute("""
  DROP SECRET IF EXISTS secret3;
  CREATE SECRET secret3 (
    TYPE S3,
    PROVIDER CREDENTIAL_CHAIN,
    CHAIN 'sso'
  );
            """)

In [20]:
con.execute(f"""
  CREATE TABLE IF NOT EXISTS t1 as 
  SELECT *
    FROM read_parquet('{CC_INDEX_S3_URL}');
  DESCRIBE t1;
""").df()

IOException: IO Error: Could not write file "/Users/praveen/Library/Mobile Documents/com~apple~CloudDocs/git/wk/git/crawl-works/cc_index.duckdb": No space left on device

In [9]:
#!cut -d' ' -f3- ../cc/cdx-00000 > ../cc/cdx-00000.jsonl 

# IGNORE EVERYTHING BELOW THIS LINE

In [ ]:
!duckdb -c "\
  create table meta as SELECT * FROM '../cc/cdx-00000.jsonl'; \
  create view status_vw as Select status, count(status) as cnt from meta group by status order by cnt desc; \
  select * from meta; \
  select * from status_vw; \
" {DB_FILE}

In [ ]:
!duckdb -c ".databases" {DB_FILE}
#!duckdb -c ".open hello.db"

In [ ]:
!duckdb -c " \
create table metastats as select trim(status) as tstatus, count(trim(status)) as cstatus, round(count(*) * 100.0/sum(count(*)) over(), 2) as pct from meta group by tstatus order by cstatus desc; \
select * from metastats; \
" {DB_FILE}

In [ ]:
!duckdb -c " \
create table metastats_norobotstxt as select trim(status) as tstatus, count(trim(status)) as cstatus, round(count(*) * 100.0/sum(count(*)) over(), 2) as pct from meta where url not like '%robots.txt' group by tstatus order by cstatus desc; \
select * from metastats_norobotstxt; \
" {DB_FILE}

In [ ]:
#!duckdb -c "drop table metastats" {DB_FILE}

In [ ]:
first_line = ""
with open("../cc/cdx-00000", "r") as file:
    first_line = file.readline()

print(first_line)

In [ ]:
!echo "{first_line}" | cut -d' ' -f3- 

In [ ]:
import json
s = '{"url": "http://1.0.0.0/robots.txt", "mime": "text/html", "mime-detected": "text/html", "status": "403", "digest": "UNFBLUUDKY6EPX25BG4X7TI6J3IS2BBZ", "length": "2706", "offset": "890", "filename": "crawl-data/CC-MAIN-2024-42/segments/1727944253858.36/robotstxt/CC-MAIN-20241011230131-20241012020131-00456.warc.gz"}'
json.loads(s)